In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 $('div.prompt').hide();
 } else {
 $('div.input').show();
$('div.prompt').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Code Toggle"></form>''')

In [2]:
from IPython.display import HTML

HTML('''
<a href="{{ site.links.github }}/raw/master/benchmarks/benchmark3.ipynb"
   download>
<button type="submit">Download Notebook</button>
</a>
''')

# Benchmark Problem 3: Dendritic Growth

In [4]:
from IPython.display import HTML

HTML('''{% include jupyter_benchmark_table.html num="[3]" revision=1 %}''')

See the journal publication entitled ["Phase Field Benchmark Problems for Dendritic Growth and Linear Elasticity"][nolink] for more details about the benchmark problems. Furthermore, read [the extended essay][benchmarks] for a discussion about the need for benchmark problems.

[benchmarks]: ../
[nolink]: https://pages.nist.gov/chimad-phase-field/

## Overview

Dendritic growth simulations are highly sensitive to the particular phase field formulation and numerical implementation employed making it useful as a benchmark problem (see, for example, [[1][karma1998quantitative]],[[2][warren1995prediction]]). Historically, dendritic growth was one of the first applications of phase field modeling \cite{Fix,Langer}, and remains a significant area of research today.  Previous analyses of both the sharp \cite{Langer,collins1985diffuse,kobayashi1993modeling,mcfadden1993phase,wang1993thermodynamically} and thin interface limits \cite{karma1996phase,karma1998quantitative,mcfadden2000thin,karma2001phase} have demonstrated that the diffuse-interface phase field formulation is asymptotically equivalent to the sharp-interface Stefan formulation. In 2001, the introduction of an "anti-trapping current" to correct for solute trapping due to the jump in chemical potential at the solid/liquid interface \cite{karma2001phase,echebarria2004quantitative} facilitated quantitative phase field modeling of alloy solidification using unphysically large diffuse interface widths.  Today, massive increases in computing power and the advent of scientific computing on graphical processing units (GPUs) enable large-scale, quantitative 3D phase field simulations of growing dendrites (see, for example, Refs.~\cite{hotzer2015large,shibuta2015solidification} and reviews \cite{takaki2014phase,granasy2014phase}).

[karma1998quantitative]: https://doi.org/10.1103/PhysRevE.57.4323
[warren1995predictio]: https://doi.org/10.1016/0956-7151(94)00285-P
[Fix]: https://doi.org/10.1007/978-3-0348-7893-7
[Langer]: https://doi.org/10.1142/9789814415309_0005
[collins1985diffuse]: https://doi.org/10.1103/PhysRevB.31.6119
[kobayashi1993modeling]: https://doi.org/10.1016/0167-2789(93)90120-P
[mcfadden1993phase]: https://doi.org/10.1103/PhysRevE.48.2016
[wang1993thermodynamically]: https://doi.org/10.1016/0167-2789(93)90189-8
[karma1996phase]: https://doi.org/10.1103/PhysRevE.53.R3017
[karma1998quantitative]:
[mcfadden2000thin]:
[karma2001phase]:
[echebarria2004quantitative]:
[hotzer2015large]:
[shibuta2015solidification]:
[takaki2014phase]:
[granasy2014phase]:


# References

